In [ ]:
# Needed if running on Colab, comment out if in local environment!
#!pip3 install open-spiel
#!pip3 install torch

: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from go_search_problem import GoProblem, GoState
from heuristic_go_problems import GoProblemLearnedHeuristic, GoProblemSimpleHeuristic
from agents import GreedyAgent, RandomAgent, AlphaBetaAgent
import matplotlib.pyplot as plt
from tqdm import tqdm
from game_runner import GameRunner
import pickle

torch.set_default_tensor_type(torch.FloatTensor)

In [ ]:
print(GoProblem)
print(GoState)
print(GreedyAgent)
print(RandomAgent)
print(AlphaBetaAgent)
print(GoProblemSimpleHeuristic)
print(GoProblemLearnedHeuristic)

In [ ]:
def load_dataset(path: str):
    with open(path, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

# We've provided a dataset with pyspiel and without (i.e., pygo)
# dataset_5x5 = load_dataset('dataset_5x5.pkl')
dataset_5x5 = load_dataset('dataset_5x5_pygo.pkl')
print("type:", type(dataset_5x5))

In [ ]:
def save_model(path: str, model):
    """
    Save model to a file
    Input:
        path: path to save model to
        model: Pytorch model to save
    """
    torch.save({
        'model_state_dict': model.state_dict(),
    }, path)

def load_model(path: str, model):
    """
    Load model from file

    Note: you still need to provide a model (with the same architecture as the saved model))

    Input:
        path: path to load model from
        model: Pytorch model to load
    Output:
        model: Pytorch model loaded from file
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

# Task 1: Convert GameState to Features

In [ ]:
def get_features(game_state: GoState):
    """
    Map a game state to a list of features.

    Some useful functions from game_state include:
        game_state.size: size of the board
        get_pieces_coordinates(player_index): get coordinates of all pieces of a player (0 or 1)
        get_pieces_array(player_index): get a 2D array of pieces of a player (0 or 1)
        
        get_board(): get a 2D array of the board with 4 channels (player 0, player 1, empty, and player to move). 4 channels means the array will be of size 4 x n x n
    
        Descriptions of these methods can be found in the GoState

    Input:
        game_state: GoState to encode into a fixed size list of features
    Output:
        features: list of features
    """
    features = [] #Logic for discerning difference between game_state being a dict vs GoState was determined in TA hours
    if isinstance(game_state, dict):
        board = game_state["board"]        
        return board.flatten().tolist()    
    board = game_state.get_board()         
    return board.flatten().tolist() 


In [ ]:
# Print information about first data point
data_point = dataset_5x5[0]
features = get_features(data_point[0])
action = data_point[1]
result = data_point[2]
print(data_point[0])
print("features", features)
print("Action #", action)
print("Game Result", result)

# Task 2: Supervised Learning of a Value Network

In [ ]:
class ValueNetwork(nn.Module):
    def __init__(self, input_size):
      super(ValueNetwork, self).__init__()
      output_size = 1

      hidden_size = 100
      self.fc1 = nn.Linear(input_size, hidden_size)
      self.fc2 = nn.Linear(hidden_size, hidden_size)
      self.linear = nn.Linear(hidden_size, output_size)
      

    def forward(self, x):
      """
      Run forward pass of network

      Input:
        x: input to network
      Output:
        output of network
      """
      x = x.float()                 
      x = torch.relu(self.fc1(x))   
      x = torch.relu(self.fc2(x))    
      return self.linear(x)     

In [ ]:
# This will not produce meaningful outputs until trained, but you can test for syntax errors
features_tensor = torch.Tensor(features)
value_net = ValueNetwork(len(features))
print("predicted Value", value_net(features_tensor))

In [ ]:
def train_value_network(dataset, num_epochs, learning_rate):
    """
    Train a value network on the provided dataset.

    Input:
        dataset: list of (state, action, result) tuples
        num_epochs: number of epochs to train for
        learning_rate: learning rate for gradient descent
    Output:
        model: trained model
    """
    # Make sure dataset is shuffled for better performance
    random.shuffle(dataset)
    # You may find it useful to create train/test sets to better track performance/overfit/underfit
    example_state = dataset[0][0]
    example_features = get_features(example_state)
    input_size = len(example_features)
    model = ValueNetwork(input_size)

    loss_function = nn.MSELoss()

    # You can use Adam, which is stochastic gradient descent with ADAptive Momentum
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    batch_size = 32

    for e in range(num_epochs):
        batch_loss = 0
        batch_counter = 0
        for data_point in dataset:
            state = data_point[0]
            features = get_features(state)
            features_tensor = torch.tensor(features)

            label_value = data_point[2]
            label = torch.tensor([[label_value]], dtype=torch.float32)

            prediction = model(features_tensor)

            loss = loss_function(prediction, label)
            batch_loss += loss
            batch_counter += 1
            if batch_counter % batch_size == 0:
                batch_loss.backward()

                optimizer.step()
                optimizer.zero_grad()
                batch_loss = 0

    return model

value_model = train_value_network(dataset_5x5, 10, 1e-4)
save_model("value_model.pt", value_model)

## Comparing Learned Value function against other Agents

In [ ]:
class GoProblemLearnedHeuristic(GoProblem):
    def __init__(self, model=None, state=None):
        super().__init__(state=state)
        self.model = model

    def encoding(self, state):
        """
        Get encoding of state (convert state to features)
        Note, this may call get_features() from Task 1. 

        Input:
            state: GoState to encode into a fixed size list of features
        Output:
            features: list of features
        """
        feat = get_features(state)
        return feat

    def heuristic(self, state, player_index):
        """
        Return heuristic (value) of current state

        Input:
            state: GoState to encode into a fixed size list of features
            player_index: index of player to evaluate heuristic for
        Output:
            value: heuristic (value) of current state
        """
        features = self.encoding(state)
        x = torch.tensor(features, dtype=torch.float32)
        with torch.no_grad():
            pred = self.model(x).squeeze().item()
        if player_index == 1:
            pred = -pred
        val = pred
        return val

    def __str__(self) -> str:
        return "Learned Heuristic"


def create_value_agent_from_model():
    """
    Create agent object from saved model. This (or other methods like this) will be how your agents will be created in gradescope and in the final tournament.
    """

    mod_path = "value_model.pt"
    feat_size = 100
    mod = load_model(mod_path, ValueNetwork(feat_size))
    heuristic_sp = GoProblemLearnedHeuristic(mod)
    learn_agent = GreedyAgent(heuristic_sp)
    return learn_agent

learned_agent = create_value_agent_from_model()
agent2 = GreedyAgent(GoProblemSimpleHeuristic())
print("Greedy Agent", agent2)
print("Learned Agent", learned_agent)

game_runner = GameRunner()
game_runner.play_tournament(learned_agent, agent2, num_games=100)